In [16]:
grammar_str11 = ("Z ::= E\n"
                 "E ::= T | E + T\n"
                "T ::= F | T * F\n"
                "F ::= ( E ) | i\n")
from extract_grammar import GrammaticalQuadrupleExtraction
# 示例用法
extractor = GrammaticalQuadrupleExtraction()
grammar_string = grammar_str11

print("原始文法: \n" + grammar_string)
non_terminals, terminals, productions, start = extractor.extract_grammar_components(grammar_string)


原始文法: 
Z ::= E
E ::= T | E + T
T ::= F | T * F
F ::= ( E ) | i



In [17]:
print("非终结符: ", non_terminals)
print("终结符: ", terminals)
print("产生式: ", productions)
print("开始符号: ", start)

非终结符:  ['Z', 'E', 'T', 'F']
终结符:  ['+', '*', '(', ')', 'i']
产生式:  {'Z': [['E']], 'E': [['T'], ['E', '+', 'T']], 'T': [['F'], ['T', '*', 'F']], 'F': [['(', 'E', ')'], ['i']]}
开始符号:  Z


In [27]:
def firstvt(grammar, non_terminals, terminals):
    first = {nt: set() for nt in non_terminals}

    def add_first(nt):
        for production in grammar[nt]:
            first_symbol = production[0]

            if first_symbol in terminals:
                first[nt].add(first_symbol)
            elif first_symbol in non_terminals:
                add_first(first_symbol)
                first[nt] |= first[first_symbol]

    for nt in non_terminals:
        add_first(nt)

    return first

# 测试数据
firstvt_set = firstvt(productions, non_terminals, terminals)

# 打印结果
for nt in non_terminals:
    print(f"FirstVT({nt}): {firstvt_set[nt]}")


RecursionError: maximum recursion depth exceeded in comparison

In [25]:
def firstvt(grammar, nonterminals, terminals):
    first = {nt: set() for nt in nonterminals}
    stack = []

    def add_first(nt, symbol):
        stack.append((nt, symbol))
        while stack:
            current_nt, current_symbol = stack.pop()
            if current_symbol in terminals:
                first[current_nt].add(current_symbol)
            elif current_symbol in nonterminals:
                for prod in grammar[current_symbol]:
                    stack.append((current_nt, prod[0]))

    for nt in nonterminals:
        for prod in grammar[nt]:
            add_first(nt, prod[0])

    return first

def lastvt(grammar, nonterminals, terminals):
    last = {nt: set() for nt in nonterminals}
    stack = []

    def add_last(nt, symbol):
        stack.append((nt, symbol))
        while stack:
            current_nt, current_symbol = stack.pop()
            if current_symbol in terminals:
                last[current_nt].add(current_symbol)
            elif current_symbol in nonterminals:
                for prod in grammar[current_symbol]:
                    stack.append((current_nt, prod[-1]))

    for nt in nonterminals:
        for prod in grammar[nt]:
            add_last(nt, prod[-1])

    return last

def operator_precedence_table(nonterminals, terminals, productions):
    table = {t1: {t2: '' for t2 in terminals} for t1 in terminals}

    first = firstvt(productions, nonterminals, terminals)
    last = lastvt(productions, nonterminals, terminals)

    for nt, rules in productions.items():
        for rule in rules:
            for i in range(len(rule) - 1):
                if rule[i] in terminals and rule[i + 1] in terminals:
                    table[rule[i]][rule[i + 1]] = '='
                if rule[i] in terminals and rule[i + 1] in nonterminals:
                    for t in first[rule[i + 1]]:
                        table[rule[i]][t] = '<'
                if rule[i] in nonterminals and rule[i + 1] in terminals:
                    for t in last[rule[i]]:
                        table[t][rule[i + 1]] = '>'

    return table

nonterminals = ['Z', 'E', 'T', 'F']
terminals = ['+', '*', '(', ')', 'i']
productions = {
    'Z': [['E']],
    'E': [['T'], ['E', '+', 'T']],
    'T': [['F'], ['T', '*', 'F']],
    'F': [['(', 'E', ')'], ['i']]
}

table = operator_precedence_table(nonterminals, terminals, productions)

# Display the operator precedence table
print("Operator Precedence Table:")
print("  ", " ".join(terminals))
for t1 in terminals:
    print(t1, " ".join(table[t1][t2] if table[t1][t2] else "." for t2 in terminals))


KeyboardInterrupt: 

In [ ]:
from collections import defaultdict

# 初始化FIRSTVT和LASTVT集
FIRSTVT = defaultdict(set)
LASTVT = defaultdict(set)

# 计算FIRSTVT集
def compute_FIRSTVT():
    for lhs in non_terminals:
        for production in productions[lhs]:
            if production[0] in terminals:
                FIRSTVT[lhs].add(production[0])
            elif len(production) > 1 and production[1] in terminals:
                FIRSTVT[lhs].add(production[1])
    
    change = True
    while change:
        change = False
        for lhs in non_terminals:
            for production in productions[lhs]:
                for symbol in production:
                    if symbol in non_terminals:
                        new_elements = FIRSTVT[symbol] - FIRSTVT[lhs]
                        if new_elements:
                            FIRSTVT[lhs].update(new_elements)
                            change = True

# 计算LASTVT集
def compute_LASTVT():
    for lhs in non_terminals:
        for production in productions[lhs]:
            if production[-1] in terminals:
                LASTVT[lhs].add(production[-1])
            elif len(production) > 1 and production[-2] in terminals:
                LASTVT[lhs].add(production[-2])
    
    change = True
    while change:
        change = False
        for lhs in non_terminals:
            for production in productions[lhs]:
                for symbol in reversed(production):
                    if symbol in non_terminals:
                        new_elements = LASTVT[symbol] - LASTVT[lhs]
                        if new_elements:
                            LASTVT[lhs].update(new_elements)
                            change = True

compute_FIRSTVT()
compute_LASTVT()

print("FIRSTVT 集:")
for nt in non_terminals:
    print(f"{nt}: {FIRSTVT[nt]}")

print("\nLASTVT 集:")
for nt in non_terminals:
    print(f"{nt}: {LASTVT[nt]}")


In [21]:
# 定义优先关系表
priority_table = {
    '+': {'+': '>', '*': '<', '(': '<', ')': '>', 'i': '<', '#': '>'},
    '*': {'+': '>', '*': '>', '(': '<', ')': '>', 'i': '<', '#': '>'},
    '(': {'+': '<', '*': '<', '(': '<', ')': '=', 'i': '<', '#': ''},
    ')': {'+': '>', '*': '>', '(': '', ')': '>', 'i': '', '#': '>'},
    'i': {'+': '>', '*': '>', '(': '', ')': '>', 'i': '', '#': '>'},
    '#': {'+': '<', '*': '<', '(': '<', ')': '', 'i': '<', '#': ''},
}

# 终结符和非终结符
terminals = ['+', '*', '(', ')', 'i']
non_terminals = ['Z', 'E', 'T', 'F']

# 产生式
productions = {
    'Z': [['E']],
    'E': [['T'], ['E', '+', 'T']],
    'T': [['F'], ['T', '*', 'F']],
    'F': [['(', 'E', ')'], ['i']]
}

# 定义优先关系获取函数
def get_priority(a, b):
    return priority_table[a][b]

# 定义规约函数
def reduce(stack):
    if stack[-1] == 'i':
        stack.pop()
        stack.append('F')
        print(f"规约: i -> F")
    elif stack[-1] == 'F' and stack[-2] == '*' and stack[-3] == 'T':
        stack.pop()
        stack.pop()
        stack.pop()
        stack.append('T')
        print(f"规约: T * F -> T")
    elif stack[-1] == 'T' and stack[-2] == '+' and stack[-3] == 'E':
        stack.pop()
        stack.pop()
        stack.pop()
        stack.append('E')
        print(f"规约: E + T -> E")
    elif stack[-1] == 'F':
        stack.pop()
        stack.append('T')
        print(f"规约: F -> T")
    elif stack[-1] == 'T':
        stack.pop()
        stack.append('E')
        print(f"规约: T -> E")
    elif stack[-1] == ')' and stack[-2] == 'E' and stack[-3] == '(':
        stack.pop()
        stack.pop()
        stack.pop()
        stack.append('F')
        print(f"规约: ( E ) -> F")

# 分析器
def operator_precedence_parser(input_string):
    stack = ['#']
    input_string += '#'
    index = 0

    while True:
        top = stack[-1]
        current = input_string[index]

        if top in terminals:
            priority = get_priority(top, current)
        else:
            priority = '<'

        if priority == '<' or priority == '=':
            stack.append(current)
            index += 1
        elif priority == '>':
            reduce(stack)
        else:
            raise ValueError(f"Invalid priority between {top} and {current}")

        print(f"栈: {stack}, 输入: {input_string[index:]}")

        if stack == ['#', 'E'] and input_string[index] == '#':
            break
        if index >= len(input_string):
            break

    # 检查是否接受
    if stack == ['#', 'E']:
        print("输入串被接受")
    else:
        print("输入串被拒绝")

# 示例用法
input_string = "i+i*i"
operator_precedence_parser(input_string)


栈: ['#', 'i'], 输入: +i*i#
规约: i -> F
栈: ['#', 'F'], 输入: +i*i#
栈: ['#', 'F', '+'], 输入: i*i#
栈: ['#', 'F', '+', 'i'], 输入: *i#
规约: i -> F
栈: ['#', 'F', '+', 'F'], 输入: *i#
栈: ['#', 'F', '+', 'F', '*'], 输入: i#
栈: ['#', 'F', '+', 'F', '*', 'i'], 输入: #
规约: i -> F
栈: ['#', 'F', '+', 'F', '*', 'F'], 输入: #
栈: ['#', 'F', '+', 'F', '*', 'F', '#'], 输入: 
输入串被拒绝


RecursionError: maximum recursion depth exceeded in comparison